In [ ]:
# default_exp foregrounds

# Foregrounds
> Load extragalactic, galactic and CMB maps
- toc: True

In [ ]:
# export

import healpy as hp
import numpy as np

from path import Path
import logging as log

from s4refsimtool.core import get_telescope, base_folder

In [ ]:
# export

emission_naming = dict(
    foreground_emission="foreground",
    CMB_unlensed="cmb-unlensed",
    CMB_lensing_signal="cmb-lensing",
    CMB_tensor_to_scalar_ratio="cmb-tensors",
)

## Sky emission

In [ ]:
# exports


def load_sky_emission(config_sky_emission, site, channel):
    """Load foreground maps for a channel

    Parameters
    ----------
    config_sky_emission : dict
        CMB-S4 sky emission configuration,
        generally config['sky_emission'] for a configuration
        loaded from a TOML file
        dictionary with standard emission names and their weights
    site : str
        'Pole' or 'Chile', case doesn't matter
    channel : str
        Channel tag, e.g. 'MFHS1'

    Returns
    -------
    output_map : numpy array
        Output map with all emissions combined, uses nan for missing pixels
    """
    telescope = get_telescope(channel)
    nside = 512 if telescope == "SAT" else 4096
    npix = hp.nside2npix(nside)
    output_map = np.zeros((3, npix), dtype=np.float32)
    realization = 0  # foregrounds are deterministic
    for emission, weight in config_sky_emission.items():
        if weight == 0:
            log.info("Skip %s", emission)
            continue
        log.info("Processing %s", emission)

        emission_map = hp.read_map(
            Path(base_folder)
            / f"{realization:08d}"
            / f"{site.lower()}_{emission_naming[emission]}_{telescope}_{channel}_filtered_telescope_all_time_all_bmap.fits",
            (0, 1, 2),
            verbose=False,
        )
        emission_map[emission_map == hp.UNSEEN] = np.nan
        if emission == "CMB_tensor_to_scalar_ratio":
            weight /= 0.01  # r of the input maps
        output_map += emission_map * weight

    output_map /= 1e6  # uK -> K
    return output_map

In [ ]:
import toml

config = toml.load("s4_reference_design.toml")

In [ ]:
import h5py

s4 = h5py.File("cmbs4_tophat.h5", 'r')

In [ ]:
for each in s4.keys():
    print(each, s4[each].attrs["center_frequency_GHz"])

In [ ]:
log.basicConfig(level=log.INFO)

In [ ]:
## inputs

ch = "LFS1"
site = "Pole"

In [ ]:
output_map = load_sky_emission(config["sky_emission"], site, ch)

In [ ]:
assert len(output_map[output_map == hp.UNSEEN]) == 0, \
    "We should be using nan not UNSEEN to simplify summing"

In [ ]:
np.nanmin(output_map), np.nanmax(output_map)

In [ ]:
assert np.nanmax(output_map) > 1e-4 and np.nanmax(output_map) < 1e-3, "Rough unit check"

In [ ]:
%matplotlib inline

In [ ]:
hp.mollview(output_map[0], min=-1e-4, max=1e-4, unit="K", title="Foregrounds I")

In [ ]:
hp.mollview(output_map[1], min=-1e-5, max=1e-5, unit="K", title="Foregrounds Q")

In [ ]:
hp.mollview(output_map[2], min=-1e-5, max=1e-5, unit="K", title="Foregrounds U")